In [1]:
import numpy as np
import pandas as pd
import math
from ipynb.fs.full.Formulas import *

Lassi


In [2]:
class Vehicle:
    def __init__(self, x, y, velocity):
        self.x = x
        self.y = y
        self.velocity = velocity
        
    def calculate_position(self,time):
        current_x=self.x+(self.velocity*time)
        return current_x, self.y

class UAV:
    def __init__(self,x,y,height,velocity,theta,radius=500):
        self.x = x
        self.y = y
        self.height = height
        self.radius = radius
        self.velocity = velocity
        self.theta = theta

    def calculate_cords(self,time):
        angle = (self.velocity*time)%(2*math.pi)
        current_x=self.x+(self.radius*math.cos(angle))
        current_y=self.y+(self.radius*math.sin(angle))
        return current_x,current_y 


In [25]:
class Relay:    

    def isbetween(self,current_x,vehicle1_x,vehicle2_x):
        left=min(vehicle1_x,vehicle2_x)
        right=max(vehicle1_x,vehicle2_x)
        return current_x > left and current_x < right
        
    def no_of_obstacles(self,vehicle1,vehicle2,time):
        vehicle1_x,vehicle1_y=vehicle1.calculate_position(time)
        vehicle2_x,vehicle2_y=vehicle2.calculate_position(time)
        ans=0
        for vehicle in list_of_vehicles:
            current_x,current_y=vehicle.calculate_position(time)
            if current_y == vehicle1_y and self.isbetween(current_x,vehicle1_x,vehicle2_x):
                ans+=1
        return ans    
    
    def determine_relay_type(self,vehicle1,vehicle2,time):
        vehicle1_x,vehicle1_y=vehicle1.calculate_position(time)
        vehicle2_x,vehicle2_y=vehicle2.calculate_position(time)
        
        if vehicle1_y == vehicle2_y:    #Sender and Receiver on same lane
            obstacles = self.no_of_obstacles(vehicle1,vehicle2,time) #Method TBD
            if obstacles == 0:
                return 'dont_relay'
            elif obstacles == 1:
                return 'same_lane_relay'
            else:
                return 'adjacent_lane_relay'
        elif abs(vehicle1_y - vehicle2_y) == 1: # Sender and Receiver on adjacent lanes
            return 'dont_relay'
        else:     # Sender and Receiver 
            return 'middle_lane_relay'

    # Vehicle 1 =sender and Vehicle 2=receiver
    
    def candidate_relay_set_middle_lane(self,vehicle1,vehicle2,time):
        candidate_relays=set()
        vehicle1_x_start,vehicle1_y_start=vehicle1.calculate_position(time)
        vehicle2_x_start,vehicle2_y_start=vehicle2.calculate_position(time)

        if vehicle1_y_start > vehicle2_y_start:
            return candidate_relay_set_middle_lane(vehicle2,vehicle1,time)
        
        vehicle1_x_end,vehicle1_y_end=vehicle1.calculate_position(time+slot_duration)
        vehicle2_x_end,vehicle2_y_end=vehicle2.calculate_position(time+slot_duration)
        
        for vehicle in list_of_vehicles:
            current_x_start,current_y_start=vehicle.calculate_position(time)
            current_x_end,current_y_end=vehicle.calculate_position(time+slot_duration)
            if isbetween(current_y,vehicle1_y,vehicle2_y)==0: 
                continue

            m = current_x_start - vehicle1_x_start
            n = vehicle2_x_start - current_x_start
            D1x , D1y = section_formula(vehicle1_x_start,vehicle1_y_start,vehicle2_x_start,vehicle2_y_start,m,n)
            D2x , D2y = section_formula(vehicle1_x_end,vehicle1_y_end,vehicle2_x_end,vehicle2_y_end,m,n)
            
            if current_x_end <= D1x or D2x <= current_x_start: # No overlap=No add
                continue

            start_boundary=max(current_x_start,D1x)
            end_boundary = min(current_x_end, D2x)
            epsilon = (end_boundary-start_boundary) / (current_x_end - current_x_start) 
            if epsilon >= 0.5:
                candidate_relays.add(vehicle)        
            
        # If candidate_relays.length()==0 till now, it means there's no obstacle in middle lanes and no need to relay. so dont add uav and return empty set
        if len(candidate_relays):
            candidate_relays=candidate_relays.union(get_uav(vehicle1,vehicle2))
        return candidate_relays

    def candidate_relay_set_single_same_lane(self,vehicle1,vehicle2,time):
        candidate_relays=set()
        vehicle1_x,vehicle1_y=vehicle1.calculate_position(time)
        vehicle2_x,vehicle2_y=vehicle2.calculate_position(time)
        for vehicle in list_of_vehicles:
            current_x,current_y=vehicle.calculate_position(time)
            if isbetween(current_x,vehicle1_x,vehicle2_x):
                candidate_relays.add(vehicle)
                break
        candidate_relays=candidate_relays.union(get_uav(vehicle1,vehicle2))
        return candidate_relays

    def candidate_relay_set_multiple_same_lane(self,vehicle1,vehicle2,time):
        candidate_relays=set()
        vehicle1_x_start,vehicle1_y_start=vehicle1.calculate_position(time)
        vehicle2_x_end,vehicle2_y_end=vehicle2.calculate_position(time+slot_duration)
        D3x=vehicle1_x_start
        D4x=vehicle2_x_end
        
        for vehicle in list_of_vehicles:
            current_x_start,current_y_start=vehicle.calculate_position(time)
            current_x_end,current_y_end=vehicle.calculate_position(time+slot_duration)
            
            if current_y==(vehicle1_y-1) or current_y==(vehicle1_y+1):
                if current_x_end <= D3x or D4x <= current_x_start:
                    continue
                
                start_boundary=max(current_x_start, D3x)
                end_boundary=min(current_x_end, D4x)
                epsilon=(end_boundary-start_boundary)/(current_x_end-current_x_start)
                
                if epsilon>=0.5:
                    candidate_relays.add(vehicle)
                    
        candidate_relays=candidate_relays.union(get_uav(vehicle1,vehicle2))
        return candidate_relays

    def get_uav(self,vehicle1,vehicle2,time):
        candidate_uavs=set()
        vehicle1_x,vehicle1_y=vehicle1.calculate_position(time)
        vehicle2_x,vehicle2_y=vehicle2.calculate_position(time)
        for uav in candidate_uavs:
            current_x,current_y=uav.calculate_cords(time)
            distance_s = ((current_x-vehicle1_x)**2) + ((current_y-vehicle1_y)**2)
            distance_s = math.sqrt(distance_s)
            distance_r = ((current_x-vehicle2_x)**2) + ((current_y-vehicle2_y)**2)
            distance_r = math.sqrt(distance_r)
            if distance_s <= 500 and distance_r <= 500:
                candidate_uavs.add(uav)
        return candidate_uavs
        

In [12]:
# Declare it in more proper manner
list_of_vehicles = [
        Vehicle(10, 1, 30),    
        Vehicle(15, 2, 35),    
        Vehicle(20, 3, 40),    
        Vehicle(25, 1, 45),    
        Vehicle(30, 2, 50),    
        Vehicle(35, 3, 55),    
        Vehicle(40, 1, 60),    
        Vehicle(45, 2, 65),    
        Vehicle(50, 3, 70),    
        Vehicle(55, 1, 75)     
]

# Creating a list of UAV objects
list_of_uavs = [
        UAV(10, 1, 200, 4, math.pi/4),
        UAV(500, 2, 150, 10, math.pi/3),
        UAV(200, 3, 250, 7, math.pi/6)
]

# Create an empty list to store flows
flows = []

# Loop through the list and form pairs
for i in range(len(list_of_vehicles)):
    for j in range(i+1, len(list_of_vehicles)):  # Ensure unique pairs
        flows.append([list_of_vehicles[i], list_of_vehicles[j]])

# Convert the list of pairs to a NumPy array
flows = np.array(flows)

# Print the 2D NumPy array
# for i in flows:
#     for j in i:
#         x= str(j.x) + " " + str(j.y)
#         print(x)

# Edging

In [31]:
temp= Relay()
for i in range(number_of_time_slots):
    for j in flows:
        print("Sender:",j[0].x,j[0].y,"Receiver:",j[1].x,j[1].y)
        print(temp.determine_relay_type(j[0],j[1],i*slot_duration))



Sender: 10 1 Receiver: 15 2
dont_relay
Sender: 10 1 Receiver: 20 3
middle_lane_relay
Sender: 10 1 Receiver: 25 1
dont_relay
Sender: 10 1 Receiver: 30 2
dont_relay
Sender: 10 1 Receiver: 35 3
middle_lane_relay
Sender: 10 1 Receiver: 40 1
same_lane_relay
Sender: 10 1 Receiver: 45 2
dont_relay
Sender: 10 1 Receiver: 50 3
middle_lane_relay
Sender: 10 1 Receiver: 55 1
adjacent_lane_relay
Sender: 15 2 Receiver: 20 3
dont_relay
Sender: 15 2 Receiver: 25 1
dont_relay
Sender: 15 2 Receiver: 30 2
dont_relay
Sender: 15 2 Receiver: 35 3
dont_relay
Sender: 15 2 Receiver: 40 1
dont_relay
Sender: 15 2 Receiver: 45 2
same_lane_relay
Sender: 15 2 Receiver: 50 3
dont_relay
Sender: 15 2 Receiver: 55 1
dont_relay
Sender: 20 3 Receiver: 25 1
middle_lane_relay
Sender: 20 3 Receiver: 30 2
dont_relay
Sender: 20 3 Receiver: 35 3
dont_relay
Sender: 20 3 Receiver: 40 1
middle_lane_relay
Sender: 20 3 Receiver: 45 2
dont_relay
Sender: 20 3 Receiver: 50 3
same_lane_relay
Sender: 20 3 Receiver: 55 1
middle_lane_rela